In [1]:
## Run in python 2.7
# Script for all sensor data

import sys

import arable
from   arable.client     import ArableClient 
a = ArableClient()
from   sensor_includes   import email, password_ccber, tenant_ccber

a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

import re
import matplotlib.pyplot as plt
import numpy             as np
import matplotlib.dates  as mdates
import altair            as alt

# Enable altair usage in Jupyter Notebook
alt.renderers.enable('notebook')

from   datetime          import timedelta
from   datetime          import datetime

dt = datetime.now()

import pandas            as     pd
from   pandas.plotting   import register_matplotlib_converters

register_matplotlib_converters()

from   io                import StringIO

import arable_data_dict  as dd

# Datetime object to string with format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append string with format "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverses dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

# Datetime object to string with format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to string with format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append string with format "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temporary lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)

def arable_query(s, b, c, d, e, f, g):
    df = a.query( select = str(s),
                  format = str(b),
                 devices = c,
                 measure = str(d),
                   order = str(e),
                     end = f,
                   start = g,
                   limit = 5000) #100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    return df


# sensors = [str(i['name']) for i in a.devices]
# Try this instead of the for loop?
sensors = []
for i in a.devices():
    sensors.append(str(i['name']))

print(sensors)

known_sensors        = ['A000522', 'A000536', 'A000502', 'A000563', 'A000507', 'A000229', 'A000513', 'A000465', 'A000318', 'A000369', 'A000466', 'A000546', 'A000514', 'A000417', 'A000461', 'A000543', 'A001512', 'A000502']
deprecated_sensors   = ['A000514', 'A000417', 'A000543', 'A001512', 'A000461']
known_active_sensors = list(set(known_sensors) - set(deprecated_sensors))
known_active_sensors = sorted(known_active_sensors, key=lambda x: int(re.search(r'\d+$',x).group()))

new_active_sensors = list(set(sensors) - set(known_sensors))

new_active_sensors = sorted(new_active_sensors, key=lambda x: int(re.search(r'\d+$',x).group()))


# FOUND NEW SENSOR WITH ID # A000*** ... ERROR - NO DATA AVAILABLE!!
# FOUND NEW SENSOR WITH ID # A000*** ... DATA RECEIVED ... ERROR - SENSOR NOT IN RANGE!!
# FOUND NEW SENSOR WITH ID # A000*** ... DATA RECEIVED ... SENSOR ENABLED!

if len(new_active_sensors) != 0:
    for i in new_active_sensors:
        print("FOUND NEW SENSOR: ", str(i),)
        for device in a.devices():
            if str(device['name']) == i:
                # Checks sensors are in the correct location (within a specific range of latitude/longitude coordinates).
                date_created   = ymd_hms_to_dt(str(i['created'])[:19])
                date_last_seen = ymd_hms_to_dt(str(i['last_seen'])[:19])
                try:
                    df = arable_query('all','csv',[device_id],"daily","time",dt,dt_to_ymd_hms(dt - timedelta(days = 1)))
                    latitude  = float(str(df['lat' ].values.astype(float)).strip('[ ]'))
                    longitude = float(str(df['long'].values.astype(float)).strip('[ ]'))
                    print("... DATA RECEIVED")
                    if 34.3 <= latitude <= 35.07 and -120.610 <= longitude <= -119.740:
                        print("... SENSOR ENABLED!")
                        known_active_sensors.append(i)
                    else:
                        print("... ERROR - SENSOR NOT IN RANGE!!")
                        # Potentially add something that notifies user of error?
                        
                except:
                    print('... ERROR - NO DATA AVAILABLE!!')
else:
    print("NO NEW SENSORS DETECTED.")
    
print("Retrieving new data from... ")
for i in range(len(known_active_sensors)//4 + 1):
    print('\t'.join(known_active_sensors[i*4:(i+1)*4]) + '\n')

device_id           = [] # Ex. A00****
device_location     = [] # Ex. Lagoon Ice Plants
device_date_created = [] # Ex. 2019-10-10T00:00:00
device_latitude     = []
device_longitude    = []
device_flags        = []
device_info         = []
# These Location Names are added to device_location.
# The corresponding Device ID is added to device_id.


for i in a.devices():
    for j in known_active_sensors:
        if i['name'] == j:
            device_location.append(str(i['location']['name']))
            device_id.append(str(i['name']))
            flags=[]
            for k in i['flags']:
                flags.append(str(k.strip('[ ]')))
            device_flags.append(flags)
            device_date_created.append(str(i['created'])[:19])
            #device_latitude.append(float(str(df['lat'].values).strip('[ ]')))
            #device_longitude.append(float(str(df['long'].values).strip('[ ]')))
            temp = {'ID'            : str(i['name']),
                    'Location'        : str(i['location']['name']),
                    'Date Created'    : ymd_hms_to_dt(str(i['created'])[:19]),
                    'Last Seen'       : ymd_hms_to_dt(str(i['last_seen'])[:19]),
                    'Flags'           : str(flags),
                    'Battery Voltage' : str(i['batt_volt']),
                    'Battery Percent' : str(i['batt_pct']),
                    'Signal Strength' : str(i['signal_strength']),
                    'State'           : str(i['state']),
                    'n'               : len(device_info) + 1}
            device_info.append(temp)

# Converts each item in device_id to a list.          
device_id              = list(map(lambda el:[el], device_id))
device_date_created_dt = [ymd_hms_to_dt(i) for i in device_date_created]

print(['{:<7}'.format(str(i)) for i in device_id])

device_info = pd.DataFrame(device_info)
device_info.to_csv(str('Sensor-Info_' + str(dt_to_ymd(dt)) + '.txt'), sep = ",")
print(device_info)

['A000465', 'A000229', 'A000563', 'A000536', 'A000318', 'A000466', 'A000369', 'A000522', 'A000546', 'A000507', 'A000513', 'A000502', 'A001512', 'A000514', 'A000417', 'A000461', 'A000543']
NO NEW SENSORS DETECTED.
Retrieving new data from... 
A000229	A000318	A000369	A000465

A000466	A000502	A000507	A000513

A000522	A000536	A000546	A000563



["['A000465']", "['A000229']", "['A000563']", "['A000536']", "['A000318']", "['A000466']", "['A000369']", "['A000522']", "['A000546']", "['A000507']", "['A000513']", "['A000502']"]
   Battery Percent Battery Voltage        Date Created  \
0            100.0            4140 2019-02-05 21:44:10   
1             48.0            3811 2019-02-05 23:19:19   
2            100.0            4147 2019-03-18 22:04:27   
3            100.0            4153 2019-02-06 00:43:30   
4             96.0            4086 2019-02-05 23:45:25   
5            100.0            4148 2019-03-12 22:47:11   
6            100.0            4150 2019-03-12 23:12:28   
7           